In [ ]:
import sympy as sp
import pandas as pd

class DiophantineSolver:
    def __init__(self, z: int, max_n: int = 50, max_solutions_per_n: int = 10):
        self.z = z
        self.max_n = max_n
        self.max_solutions_per_n = max_solutions_per_n
        self.x, self.y = sp.symbols('x y', integer=True)
        self.a = 2 * z + 1
        self.mod_val = 16 * self.a
        self.constraint = 8 * self.a
        self.results_df = pd.DataFrame(columns=["x", "y", "n"])

    def _satisfies_constraints(self, x_val, y_val):
        return (
            x_val >= 0 and
            y_val >= 0 and
            x_val % self.mod_val == self.constraint and
            y_val % self.mod_val == self.constraint
        )

    def _solve_for_n(self, n):
        rhs = 128 * n * self.a**2
        eq = sp.Eq(self.x**2 + self.a * self.y**2, rhs)
        try:
            solutions = sp.diophantine(eq)
            count = 0
            seen = set()

            for sol in solutions:
                if len(sol) != 2:
                    continue
                x_expr, y_expr = sol
                try:
                    x_val = int(x_expr.evalf())
                    y_val = int(y_expr.evalf())
                except (TypeError, ValueError):
                    continue

                if self._satisfies_constraints(x_val, y_val) and (x_val, y_val) not in seen:
                    self.results_df.loc[len(self.results_df)] = [x_val, y_val, n]
                    seen.add((x_val, y_val))
                    count += 1

                if count >= self.max_solutions_per_n:
                    break

        except Exception as e:
            print(f"Error solving for n = {n}: {e}")

    def solve(self):
        print(f"Solving Diophantine equation for z = {self.z} → a = {self.a}")
        print(f"Constraint: x ≡ y ≡ {self.constraint} mod {self.mod_val}")

        for n in range(1, self.max_n + 1):
            self._solve_for_n(n)

        if not self.results_df.empty:
            print("\nValid solutions:")
            print(self.results_df)
        else:
            print("No valid solutions found.")

        return self.results_df

class PellSolver:
    def __init__(self, z: int):
        self.z = z
        self.A, self.B = sp.symbols('a b', integer=True)
        self.U, self.V = sp.symbols('u v', integer=True)
        self.results = {}

    def _extract_sample_solution(self, solns):
        for sol in solns:
            if all(expr.free_symbols == set() for expr in sol):
                # Already numeric
                return tuple(int(expr) for expr in sol)
            else:
                # Symbolic parametric: substitute t = 1 (or whatever param is used)
                params = set()
                for expr in sol:
                    params.update(expr.free_symbols)
                subs = {param: 1 for param in params}
                try:
                    return tuple(int(expr.subs(subs).evalf()) for expr in sol)
                except Exception:
                    continue
        return None

    def solve_pell_equation_1(self):
        rhs = 2 * self.z + 1
        eq1 = sp.Eq(self.A**2 - 2 * self.B**2, rhs)
        solns1 = sp.diophantine(eq1)
        self.results['eq1'] = self._extract_sample_solution(solns1)

    def solve_pell_equation_2(self):
        eq2 = sp.Eq(self.U**2 - 2 * self.V**2, -2)
        solns2 = sp.diophantine(eq2)
        self.results['eq2'] = self._extract_sample_solution(solns2)

    def solve(self):
        print(f"\nSolving Pell-type equations for z = {self.z}")
        self.solve_pell_equation_1()
        self.solve_pell_equation_2()

        print(f"\nFundamental solution to:")
        print(f"1. a^2 - 2b^2 = {2 * self.z + 1}")
        print("   →", self.results['eq1'] if self.results['eq1'] else "No solution found.")

        print(f"2. u^2 - 2v^2 = -2")
        print("   →", self.results['eq2'] if self.results['eq2'] else "No solution found.")

        return self.results

if __name__ == "__main__":
    z = 4  

    pell_solver = PellSolver(z)
    pell_results = pell_solver.solve()

    dio_solver = DiophantineSolver(z, max_n=200, max_solutions_per_n=10)
    dio_results = dio_solver.solve()


Solving Pell-type equations for z = 4

Fundamental solution to:
1. a^2 - 2b^2 = 9
   → (51, -36)
2. u^2 - 2v^2 = -2
   → (4, 3)
Solving Diophantine equation for z = 4 → a = 9
Constraint: x ≡ y ≡ 72 mod 144

Valid solutions:
       x    y    n
0     72   72    5
1    216   72    9
2    360   72   17
3    504   72   29
4     72  216   41
5    216  216   45
6    648   72   45
7    360  216   53
8    792   72   65
9    504  216   65
10   648  216   81
11   936   72   89
12   792  216  101
13    72  360  113
14   216  360  117
15  1080   72  117
16   936  216  125
17   360  360  125
18   504  360  137
19  1224   72  149
20  1080  216  153
21   648  360  153
22   792  360  173
23  1368   72  185
24  1224  216  185
25   936  360  197
